In [103]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold


In [64]:
sp = pd.read_csv("sphist.csv")

In [65]:
sp.head(5)

,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883


In [66]:
sp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16590 entries, 0 to 16589
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       16590 non-null  object 
 1   Open       16590 non-null  float64
 2   High       16590 non-null  float64
 3   Low        16590 non-null  float64
 4   Close      16590 non-null  float64
 5   Volume     16590 non-null  float64
 6   Adj Close  16590 non-null  float64
dtypes: float64(6), object(1)
memory usage: 907.4+ KB


In [67]:
sp['Date'] = pd.to_datetime(sp['Date'])

In [68]:
sp.sort_values(ascending=True, inplace=True, by='Date')

In [69]:
sp.reset_index(inplace=True, drop = True)

In [70]:
sp.head(5)

,Date,Open,High,Low,Close,Volume,Adj Close
0,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
1,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
2,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93
3,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98
4,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08


In [71]:
for index, data in sp.iterrows():
    if (index < 5):
        sp.loc[index, "day_5_avg_price"] = 0
    else:
        sp.loc[index, "day_5_avg_price"] = sp.iloc[index-5:index]["Close"].mean()


In [72]:
for index, data in sp.iterrows():
    if (index < 365):
        sp.loc[index, "day_365_avg_price"] = 0
    else:
        sp.loc[index, "day_365_avg_price"] = sp.iloc[index-365:index]["Close"].mean()


In [73]:
sp["ratio_5_365"] = sp["day_5_avg_price"] / sp["day_365_avg_price"]
sp.replace([np.inf, -np.inf], np.nan, inplace=True)
sp["ratio_5_365"] = sp["ratio_5_365"].fillna(0)

In [74]:
sp.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close,day_5_avg_price,day_365_avg_price,ratio_5_365
0,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000,0.000,0.0,0.0
1,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000,0.000,0.0,0.0
2,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000,0.000,0.0,0.0
3,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000,0.000,0.0,0.0
4,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000,0.000,0.0,0.0
5,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001,16.900,0.0,0.0
6,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000.0,17.090000,16.974,0.0,0.0
7,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000.0,16.760000,17.022,0.0,0.0
8,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000.0,16.670000,16.988,0.0,0.0
9,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000.0,16.719999,16.926,0.0,0.0


In [75]:
# filter rows before 1951-01-03 since the day_365_avg_price and ratio

sp = sp[sp["Date"] > datetime(year=1951, month=1, day=2)]

In [82]:
# since its already only the weekdays we need to filter futher more
sp = sp[sp["ratio_5_365"] != 0]

In [86]:
sp.shape

(16225, 10)

In [87]:
sp = sp.dropna(axis = 0)

In [88]:
sp.shape

(16225, 10)

In [89]:
train = sp[sp["Date"].dt.strftime('%Y') < "2013"].copy()
test = sp[sp["Date"].dt.strftime('%Y') >= "2013"].copy()

In [90]:
train.shape

(15486, 10)

In [91]:
test.shape


(739, 10)

In [93]:
test.head(5)

,Date,Open,High,Low,Close,Volume,Adj Close,day_5_avg_price,day_365_avg_price,ratio_5_365
15851,2013-01-02,1426.189941,1462.430054,1426.189941,1462.420044,4.202600e+09,1462.420044,1418.641992,1327.534055,1.068629
15852,2013-01-03,1462.420044,1465.469971,1455.530029,1459.369995,3.829730e+09,1459.369995,1425.793994,1327.908247,1.073714
15853,2013-01-04,1459.369995,1467.939941,1458.989990,1466.469971,3.424290e+09,1466.469971,1433.702002,1328.224877,1.079412
15854,2013-01-07,1466.469971,1466.469971,1456.619995,1461.890015,3.304970e+09,1461.890015,1443.376001,1328.557617,1.086423
15855,2013-01-08,1461.890015,1461.890015,1451.640015,1457.150024,3.601600e+09,1457.150024,1455.267993,1328.898603,1.095093


In [98]:
# train the model
model = LinearRegression()
features = ["day_5_avg_price", "day_365_avg_price", "ratio_5_365"]
target = "Close"
model.fit(train[features], train[target])
predictions = model.predict(test[features])
mse = mean_squared_error(predictions, test[target])
rmse = mse ** 0.5

In [99]:
print(rmse)

22.178149148967492


In [100]:
test["predicted_close"] = predictions

In [102]:
test.loc[:, ["day_5_avg_price", "day_365_avg_price", "ratio_5_365", "Close", "predicted_close"]].head(10)

,day_5_avg_price,day_365_avg_price,ratio_5_365,Close,predicted_close
15851,1418.641992,1327.534055,1.068629,1462.420044,1419.415645
15852,1425.793994,1327.908247,1.073714,1459.369995,1426.621558
15853,1433.702002,1328.224877,1.079412,1466.469971,1434.589960
15854,1443.376001,1328.557617,1.086423,1461.890015,1444.338283
15855,1455.267993,1328.898603,1.095093,1457.150024,1456.322190
15856,1461.460010,1329.241644,1.099469,1461.020020,1462.560736
15857,1461.180005,1329.669398,1.098905,1472.119995,1462.275062
15858,1463.730005,1330.139124,1.100434,1472.050049,1464.841583
15859,1464.846021,1330.631644,1.100865,1470.680054,1465.962522
15860,1466.604028,1331.135042,1.101770,1472.339966,1467.730477


In [111]:
# check performance

# train the model
model = LinearRegression()
features = ["day_5_avg_price", "day_365_avg_price", "ratio_5_365"]
target = "Close"
model.fit(train[features], train[target])
predictions = model.predict(train[features])
train_mse = mean_squared_error(predictions, train[target])
train_rmse = train_mse ** 0.5
train_var = predictions.var()

predictions = model.predict(test[features])
test_mse = mean_squared_error(predictions, test[target])
test_rmse = test_mse ** 0.5
test_var = predictions.var()

In [112]:
print("train_rmse is {}", train_rmse)
print("train_var is {}", train_var)
print("test_rmse is {}", test_rmse)
print("test_var is {}", test_var)

train_rmse is {} 10.50481693606088
train_var is {} 227466.51931003344
test_rmse is {} 22.178149148967492
test_var is {} 37381.49650338345
